# Airflow Lab
In this lab, you'll use a `docker-compose.yml` file to install a set of working docker images with a running airflow setup.  You'll then be able to use this to explore the Airflow UI by running and inspecting some tasks in a DAG.

Requirements:
Before starting this lab you should have installed docker to be run via command line as per previous lab.  

The lab is composed of the following sections
1. Quick view of the docker compose file
2. Run docker-compose like you did at the end of the containers lab, we'll pick up from there.
3. We'll log into the Airflow UI
4. We should now be able to see the DAG view and the samples provided by default.
5. Next we'll explore some of the views by running and inspecting some of the examples.


# 1.1 Creating a DAG skeleton
In the directory where you ran docker-compose, you should now see a new directory `dags`.  Navigate to this directory as this is where we'll build and store our DAGS.

Our process will be the following
* Create a file called `user_processing.py`
* Our first line should be `from airflow import DAG`, this is what will signal to Airflow that the file contains a DAG
* Now we can instantiate Airlflows DAG object and we'll need three parameters to start with:
    - A unique tag ID.  We just need to choose a name which will be unique to this Airflow instance.
    - A start date.  We'll import python's datetime, and create a starting date.
    - Define an interval.  There are some keywords, but essentially Airflow uses `cron` notation for the DAG
    - Set the catchup parameter.  This tells the DAG to catch up if it's missed runs when it gets started.  We'll set it to False to start with since we'll be triggering one run.
* As a first pass, we'll not pass any body to the DAG, we'll create an empty one to check our work.

Our file at this time should look like this
```python
from airflow import DAG
from datetime import datetime

with DAG('user_processing', start_date=datetime(2022,1,1), schedule_interval='@daily', catchup=False) as dag:
    None
```

This simple start is how you'll create most any DAG.  In the next step we'll use that as a starting point to create a postgres table using an airflow `provider`

# 1.2 Use a provider to add a table to the postgres database
In this section, we'll take advantage of Airflows built in providers, namely `postgres` in order to create a database table.  Airflow is build from blocks, and comes with default operators to get you started, but a LOT of tools have packages you can install to allow Airflow to interact with for example, AWS, Snowflake, and DBT.

Our steps here are
* Add the import of the PostgresOperator, it comes with Airflow so it's origin is `airflow.providers...`
* Let's now use that operator creating an instance of it in a variable, `create_table`
* The parameters we need are
    - task_id, a name for this operation, typical convention is to use the same name as the variable, thus in this case `create_table`
    - postgres_conn_id, this should point to an Airflow connection.  We will create that in the next step, so for now just give it the name `postgres`
    - sql, finally, we pass in our sql command to execute.  Our sql just creates a table with slots for some identification data.
    

```python
from airflow import DAG
from airflow.providers.postgres.operators.postgres import PostgresOperator
from datetime import datetime

with DAG('user_processing', start_date=datetime(2022,1,1), schedule_interval='@daily', catchup=False) as dag:

    create_table = PostgresOperator(
        task_id='create_table',
        postgres_conn_id='postgres',
        sql='''
            CREATE TABLE IF NOT EXISTS users (
                firstname TEXT NOT NULL,
                lastname TEXT NOT NULL,
                country TEXT NOT NULL,
                username TEXT NOT NULL,
                password TEXT NOT NULL,
                email TEXT NOT NULL
                );
                '''
    );
```

## !Note how we used 'IF NOT EXISTS'
If you'll remember from lecture, we mentioned idempotent operations.  Even in this simple case it is really important.  If the table already exists, the operation won't fail. We know we have the table created after we run this, so we could always run it again if in doubt.  If we had simply written `CREATE TABLE users...` the task would fail on any subsequent run.  We want to get in the habit of writing code that can be rerun and produce a predictable state so our pipelines can rerun.

# 1.3 Create postgres connection
In this section, we'll create the connection to the postgres database.  

From the Airflow UI...

* Go to the upper menu option `Admin` and select `Connections`
* You should see a blue box with a `+` on the left hand side, click on the `Actions` pull down next which should land you on a form to create the new connection.
* Now we'll give this connection an id, *which should match the name we used in the code*, so we'll name it `postgres`
* For Connection Type, use the pulldown to select `Postgres`
* Finally add a description to identify the purpose or any relevant information.  
* For Host, type in `postgres`
* Login is `airflow`
* Password is `airflow`
* Port is `5432`
* Now we can click the `Save` button.



# 1.4 Refresh Airflow to see the DAG
At this point, you should be able to refresh your browser showing the DAG view, and the DAG should now show up at the bottom of the page

If it doesn't, look near the top of the page.  If there were syntax errors or other parsing errors, that would prevent the DAG from loading.

In some instances, you'll need to wait a few minutes to allow Airflow to pick up the DAG from drive, typially 5 minutes should be enough.

# 1.5 Testing the DAG via docker and the Airflow scheduler
Now we'll put our linux and docker knowledge to work to test the DAG from the command line.  First, we'll look up the name of the scheduler.  Then we'll use docker to start a bash session inside that container.  From there we'll use Airflow's cli commands to do a test run of our new DAG.

Return the terminal where you executed the `docker-compose`, or open a new one to execute the following commands.

* At the command prompt execute `docker-compose ps` and note down or copy the name of the container with `scheduler` in it.
* Now we'll use access that container using the `docker exec` command.  We'll plass two flags to it, `i` for interactive and `t` for TTY.  The second flag sets up terminal access.  The first keeps the connection open.   Finally, we'll need the name of the container from the last step along with the command. Since we want a bash session, we'll do `/bin/bash`.  All together that is
    - `docker exec -it <name of scheduler> /bin/bash`
* You should now see a different prompt, as you should be in a bash session *inside* the scheduler container.
* Now type `airflow` and you should see the help and flag options scroll by.
* To test our DAG we'll need it's name along with the task.
    - The command we want to use is `tasks test`
    - our DAG name is `user_processing`
    - the task name is `create_table`
    - finally, let's pass a start date, `2022-01-01`
    - All together `airflow tasks test user_processing create_table 2022-01-01`
* If all is set up correctly, you should now see a `Marking task as SUCCESS.`

You could at this point, go back to the UI and change the password for the connection, which would cause this to fail.  You don't need to exit, simply update the connection, run the test again.  Of course, set the password back to `airflow` and watch it run before we exit.

* To exit the container use `CTL-D` and you'll be back at the original terminal.

### !NOTE!  This is a good practice to set up as a habit.   It's much easier to fix a broken task if you catch it right after create it, as opposed to as part of a larger chain of tasks.

# 1.6 Implementing a sensor
In this section, we'll add a second task to our DAG, a sensor.  It will check if an API is ready.  The procedure will follow much the same path as our first task, with some sensor specific information.

* We'll add the import statement to get access to the `HttpSensor`
* Create a task with a unique ID, and save that in a variable
* Similar to our postres connection, we'll give it an http connection ID
* Finally, we'll specify the endpoint.

```python
from airflow import DAG
from airflow.providers.postgres.operators.postgres import PostgresOperator
from airflow.providers.http.sensors.http import HttpSensor
from datetime import datetime

with DAG('user_processing', start_date=datetime(2022,1,1), schedule_interval='@daily', catchup=False) as dag:

    create_table = PostgresOperator(
        task_id='create_table',
        postgres_conn_id='postgres',
        sql='''
            CREATE TABLE IF NOT EXISTS users (
                firstname TEXT NOT NULL,
                lastname TEXT NOT NULL,
                country TEXT NOT NULL,
                username TEXT NOT NULL,
                password TEXT NOT NULL,
                email TEXT NOT NULL
                );
                '''
    )
    
    is_api_available = HttpSensor(
        task_id='is_api_available',
        http_conn_id='user_api',
        endpoint='api/'
    )
```

As we did in our previous task, we'll now create a connection to match.

# 1.7 Create a connection for the `user_api`
In this section, we'll repeat pretty much the same steps for our postgres connection but we'll use an http end point instead.

Our main differences are...
* our id is `user_api`
* connection type should be `HTTP`
* Set `Host` to `https://randomuser.me/`



# 1.8 Testing our new http connection
We can now retrace our steps as we did for the postress task and test it via the airflow api.

# 1.9 HTTP operator to extract users
In this section, we'll create the operator to extract asers from the endpoint API

Very similar procedure, import the SimpleHttpOperator and then set up the task.  The notable parameters this time around are
* http_conn_id, which shoudl be the same as the sensor that checks if the api is up and running
* method, to specify which of the HTTP operations we want, (GET, POST etc)
* response_filter, which is a function passed which we'll use to pass in a functionto process the incoming data.  The data is json formatted, so we'll convert that to a string using the python json library
* log_response, to indicate we want to log the response

Up at the top we'll need the imports
```python
from airflow.providers.http.operators.http import SimpleHttpOperator
import json
```

Our task will look like
```python
    extract_user = SimpleHttpOperator(
        task_id='extract_user',
        http_conn_id='user_api',
        endpoint='api/',
        method='GET',
        response_filter=lambda response: json.loads(response.text),
        log_response=True
    )
````

You can add that to our growing DAG file.  The order of the tasks is important, so make sure you add the `extract_user` to the end.

# 1.10 Test the new task in command line
Just like before, run the airflow cli test using the new task `extract_user`

# 1.11 Next we'll process an extracted user with python
In the last step we created a task that can pick up user info from an external api.  Now we need to process that data into a format we'll use and convert it from json.  We'll use the PythonOperator for this custom task.

As usual, we start with the import, and in addition since we're processing json we'll use pandas `json_normalize`
```python
from airflow.operators.python import PythonOperator
...
from pandas import json_normalize
```

And next we'll implement a function to use with our operator.  Note how we use `xcom_pull` to ask for the result of the previous task, `extract_user`
```python
def _process_user(task_instance):
    user = task_instance.xcom_pull(tasks_id="extract_user")
    user = user['results'][0]
    processed_user = json_normalize({
        'firstname': user['name']['first'],
        'lastname': user['name']['last'],
        'country': user['location']['country'],
        'username': user['login']['username'],
        'password': user['login']['password'],
        'email': user['email']})
    processed_user.to_csv('/tmp/processed_user.csv',
                          index=None,
                          header=False)
```

And finally, just as before, we'll use the new operator to build a task.  After the usual task_id, note how we pass the python function above as the `python_callable` parameter.
```python
    process_user = PythonOperator(
        task_id='process_user',
        python_callable=_process_user
    )
```

# 1.12 Pausing to look at our tasks in the Graph view
At this point, let's look at the Graph view to see how things are stacking up.

After updating the file and reloading the Airflow UI, go to the DAG view and click on the dag name.  Once a new window appears, click on the `Graph` view.  You'll see the tasks we've declared so far.

Note however, that they are still not related to one another. 

# 1.13 Saving the retrieved user into the database
In this section, we'll create the last part of the pipeline, and we'll use a slightly different function from the `airflow.providers.postgres` library.  Aside from the high level functionality like `PostgresOperator`, we have access to some of the underlying functionality in order to customize them.  These smaller operations are called `hooks`.  All the packages have them, they are simply smaller atomic pieces of the functionality provided.

In this case we'll import the following
```python
from airflow.providers.postgres.hooks.postgres import PostgresHook
```

Next, we'll create one helper function to use the hook to save the retrieved user info.  Well rely on the hooks method `copy_expert`, which is the reason we're accessing the hook.  It's good to get familiar with the functionality available with the hooks, as those functions are not available, in this case, the `PostgresOperator`.  After the import declarations, and before the DAG we'll add 
```python
def _store_user():
    hook = PostgresHook(postgres_conn_id='postgres')
    hook.copy_expert(
        sql="COPY users FROM stdin WITH DELIMITER as ','",
        filename='/tmp/processed_user.csv'
    )
```

Finally, we'll create our second python based task.  Very similar to the last one...
```python
    store_user = PythonOperator(
        task_id='store_user',
        python_callable=_store_user
    )
```

As usual, after update, reload the Airflow UI

# 1.14 Let's connect the tasks in the DAG
If you navigate to the Graph view, you should see all the tasks we've built up to now.  However they are no dependencies.  It's time to link them up.  Leave the Graph view open and let's add that step to our file.

At the bottom of the file, after the DAG block we'll add
```python
create_table >> is_api_available >> extract_user >> process_user >> store_user
```

Save the file and reload the Airflow UI which is open to the `Graph` page.  You should now see your tasks in a directed graph shape instead of the vertical default view.

# 1.15 Finally, let's run the DAG
From the DAG view, toggle the slider to the left of the DAG name.  Then on the right side of the page, you'll see the play (run) icon.  Click on it to activate the DAG. You should see the progression of green circles now as the DAG progresses.  

If it fails for some reason, you can go to the Graph view and the steps that succeded will be in green, the failed task should be in `red`.

If at task failed, click on the box in the Graph view, and in the popup window select `log`.  You should be able to see the cause of the failure.  After fixing it, you can run the DAG again and iterate.

# 1.16 Last but not least, let's check the database manually via the command line

Let's start a session in the worker container to see what we picked up from our mini pipeline.

At the command prompt, let's run `docker-compose ps` and get the full name of the `worker` container.

*NOTE* that if you type `docker ps` by mistake, you will see the DOCKER IDs, image names... to get the friendly name, remember to use `docker-compose ps`

Now let's start a session in the worker with 
```bash
docker exec -it materials_airflow-worker_1 /bin/bash
```

First off let's see what we picked up and stored to csv with `ls /tmp`.  We should see the file `processed_user.csv`.  A quick `cat /tmp/processed_user.csv` should display the name and details we randomly picked up.

Now let's `CTL-D` to exit this container and start a session in the database.  Just like before let's get the name from a `ps` command and execute the command
```bash
docker exec -it materials_postgres_1 /bin/bash
```

Once we are in the bash session, use `psql -Uairflow` to access the database.  You should see the prompt change to `airflow#`

Now we can execute sql at the prompt.  `SELECT * FROM users;`

And you now should be able to see the name(s) picked up and inserted into the database.

# Quick review
Contratulations if you've gotten to this point!  This put a lot of subjects together you've picked up during the course to make them happen.

* Using Docker and containers to create an environment.
* Use linux commands to probe into those containers and into a postgres database.
* Created a small pipeline in Airflow including
    - Two connections, one to a database and one to an html endpoint
    - One sensor to detect when a file is available
    - A postgres task and a hook
    - Two python tasks
    - Transfering data from one task to another
    
And on top of that you should now also feel confident going in and debugging.  You have the combination of the Airflow logs, running test runs from inside the airflow scheduler, and direct access to the database from the database container.

And although small, you've ETLd, that is to say, Extracted, Transformed and Loaded data in your small pipeline!